| [Main](index.ipynb) | [Part 2: Basic Exploration](spotify_2_basic_exploration.ipynb) >

---

# Data Scraping and Wrangling

To begin our project, we first need to collect some data. We'll scrape a large number of music tracks (and their corresponding metrics) from [Spotify](https://www.spotify.com/), using their [Web API](https://developer.spotify.com/web-api/). To interface with this API, we'll use the convenient [spotipy](https://github.com/plamere/spotipy) Python library.

We'll pull in tracks from [my personal Spotify library](https://open.spotify.com/user/nxbrnt/playlist/1ypVHzjgXq07PwQR7SBcGU?si=d56V5YZ-Q1y4FhiWzXN0gg) (~6000 tracks), and from the [Billboard Year-End Hot 100](https://en.wikipedia.org/wiki/Billboard_Hot_100) for every year since 1960 (~6000 tracks).

We'll then filter, merge, and wrangle this data into a [pandas](https://pandas.pydata.org/) dataframe that is well-suited for exploration and &mdash; later &mdash; *classification*.

*All of the prototyping and data exploration that went into the design of these scraping and wrangling functions is omitted here, but their functionality is discussed thoroughly as they are introduced below.*

---

First, let's do some standard imports, configure pandas to show more columns, and load in our API keys. To add in your own API keys, add them to keys.py, or create secret_keys.py and add them there.

In [1]:
import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util

pd.options.display.max_columns = 50

#Load Spotify API keys
from keys import *

# Authorization

Now, we need to obtain an authorization token from Spotify using spotipy. 

**Note**: The commented-out code at the top of the cell will write the contents of the cell (without executing the code) to the specified .py file (spotify_auth.py in this case). This will allow us to re-use these functions in later notebooks. An imported module can only use imports that are imported locally in said module, so we must include them explicitly when writing the file.

---
`spotify_auth` takes a dictionary of Spotify API credentials and returns a Spotify API object.

In [1]:
# %%writefile spotify_auth.py
# #Imports here for writefile magic only. Not Pythonic.
# import pandas as pd
# import spotipy
# import spotipy.util as util

def spotify_auth( auth_dict ):
    
    """Obtain a Spotify authorization token"""
    
    client_id = auth_dict['client_id']
    secret = auth_dict['secret']
    redirect_uri = auth_dict['redirect_uri']
    scope = auth_dict['scope']
    username = auth_dict['username']

    #If Chrome doesn't redirect, find redirect url using 
    #Chrome Developer Tools (Details TBA)
    token = util.prompt_for_user_token(username, scope, client_id, secret, 
                                       redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
        print('Successfully received auth token.')
    else:
        print('Cannot get token for' + username + '.')
        return
    
    return sp

# Scraping Functions

Our main scraping function - `scrape_playlist_dataframe` - wraps up a number of other functions defined below:

- `scrape_playlist_raw` pulls all tracks from the playlist of interest. While doing so, it uses the resulting track ids to pull in each track's *audio features*, *artist info*, and *album info*, each using a separate API call. This data is sculpted a bit and returned as raw lists.

- `tracks_dataframe` constructs a dataframe of tracks from the raw tracks list, and removes redundant columns.

- `features_dataframe` fills missing raw feature dictionaries, converts the duration variable from milliseconds to minutes, and returns this as a dataframe with redundant columns dropped.

- `artists_dataframe` pulls out the number of followers an artist has, and capitalizes the list of genres associated with the artist. Redundant columns are dropped and a dataframe is returned.

- `albums_dataframe` converts the album's release date (string) into `'release_year'`, `'release_month'`, and `'release_day'` numerical variables. It then returns a dataframe with redundant columns dropped.

`scrape_playlist_dataframe` then simply joins all of these dataframes and returns the result.

---

The different API requests can each return a different number of results:

**Artists: 50, Albums: 20, TracksFromPlaylist: 100, Features: 100.**

In order to pull all results in a single loop, we simply limit the number of results from all calls to the minimum (20). We could pull all of our data with roughly half the number of calls if we allowed for a different number of results from each, but this keeps our code simple.

In [3]:
# %%writefile scrape_playlist_dataframe.py
# #Imports here for writefile magic only. Not Pythonic.
# import pandas as pd
# import spotipy
# import spotipy.util as util

def scrape_playlist_raw(sp, username, playlist_id):
    
    #Scrape tracks
    results = sp.user_playlist_tracks(username, playlist_id, limit=20) 
    
    tracks = [ x['track'] for x in results['items'] ]
    
    results_ids = [ x['id'] for x in tracks ]
    features = sp.audio_features( results_ids )
    
    #This only pulls first artist id per track for now.
    results_ids_artist = [ x['artists'][0]['id'] for x in tracks ] 
    artists = sp.artists( results_ids_artist )['artists']
    results_ids_album = [ x['album']['id'] for x in tracks ]
    albums = sp.albums( results_ids_album )['albums']
    
    while results['next']:
        
        results = sp.next(results)
        
        #Pull track, audio, artist, and album info, using separate API calls
        results_tracks = [ x['track'] for x in results['items'] ]
        results_ids = [ x['id'] for x in results_tracks ]
        results_features = sp.audio_features( results_ids )
        results_ids_artist = [ x['artists'][0]['id'] for x in results_tracks ]
        results_artists = sp.artists( results_ids_artist )['artists']
        results_ids_album = [ x['album']['id'] for x in results_tracks ]
        results_albums = sp.albums( results_ids_album )['albums']
        
        tracks.extend( results_tracks )
        features.extend( results_features )
        artists.extend( results_artists )
        albums.extend( results_albums )
    
    return tracks, features, artists, albums


def tracks_dataframe(tracks):
    #Construct tracks dataframe
    df_tracks = pd.DataFrame(tracks)

    #Drop unwanted columns
    cols = ['album', 'artists', 'duration_ms', 'external_ids', 'external_urls',
            'href', 'type', 'uri'] 
    df_tracks.drop(cols, axis=1, inplace=True)
    
    return df_tracks


def features_dataframe(features, ids):
    
    """
    ids need to be specified in order to fill the 'id' field in missing dicts
    """
    
    #Fill missing feature dicts
    #This uses the first dictionary in the list as a template
    #*Assumes first dictionary isn't missing.*
    none_dict = { key:None for key in features[0] }
    for i,x in enumerate(features):
        if x is None:
            none_dict['id'] = ids[i]
            features[i] = none_dict
    
    #Construct features dataframe    
    df_features = pd.DataFrame(features)

    #Convert duration from milliseconds to minutes
    df_features['duration_mins'] = df_features['duration_ms']/60000
    
    #Drop unwanted columns
    cols = ['id', 'analysis_url', 'track_href', 'type', 'uri', 'duration_ms']
    df_features.drop(cols, axis=1, inplace=True)
    
    return df_features
    
def artists_dataframe(artists):
    #Construct tracks dataframe
    df_artists = pd.DataFrame(artists)
    
    #Pull total number of followers from 'followers' dictionary
    df_artists['num_followers'] = df_artists['followers'].transform( 
        lambda x: x['total'] )
    
    #Capitalize genre names
    def strings_to_titles(list_of_strings):
        return [string.title() for string in list_of_strings]
    df_artists['genres'] = df_artists.genres.transform( 
        lambda x: strings_to_titles(x) )
    
    #Drop unwanted columns
    cols = ['num_followers','genres','images','name','popularity','id']
    df_artists = df_artists[cols]
    
    return df_artists

def albums_dataframe(albums):
    #Construct tracks dataframe
    df_albums = pd.DataFrame(albums)
    
    #Construct integer release year column
    df_albums['release_year'] = df_albums.release_date.str[0:4].astype('int')
    
    #Float month/day columns. To allow for missing (NaN) info
    df_albums['release_month'] = pd.to_numeric( 
        df_albums.release_date.str[5:7], errors='coerce')
    df_albums['release_day'] = pd.to_numeric( 
        df_albums.release_date.str[8:10], errors='coerce')
    
    #Drop unwanted columns
    cols = ['album_type', 'images', 'label', 'name', 'popularity', 
            'release_year', 'release_month', 'release_day']
    df_albums = df_albums[cols]
    
    return df_albums
    
def scrape_playlist_dataframe(sp, username, playlist_id):
    
    #Scrape tracks, features, artists, albums as lists, derive ids
    tracks, features, artists, albums = scrape_playlist_raw(sp, username, 
                                                            playlist_id)
    ids = [ x['id'] for x in tracks ]
    
    #Build tracks and features dataframes
    df_tracks = tracks_dataframe(tracks)
    df_features = features_dataframe(features, ids)
    df_artists = artists_dataframe(artists)
    df_albums = albums_dataframe(albums)
    
    #Join into one dataframe
    df = df_tracks.join(df_features).join(df_artists, rsuffix='_artist').join(
        df_albums, rsuffix='_album')
    
    return df

The above functions allow us to construct a nicely structured dataframe from *any* Spotify playlist. In order to construct our complete dataset, we will pull a number of playlists and combine them into one dataframe.

---

- `scrape_all_playlists` scrapes a playlist full of tracks from my personal music library, as well as 6 more dataframes of Billboard Hot 100 tracks (one playlist per decade). 

*Note: These Billboard playlists were constructed *partially* by hand, so there will be some false positive or missing tracks.*

These dataframes are merged, keeping track of which dataset they belong to (the `'ds'` column) and what decade the Billboard tracks belong to (the `'decade'` column). We also add a dummy column for certain plotting functions that need it, and we remove all columns that won't be utilized in this project, including album images and preview urls.

In addition, we pull in a playlist of tracks from Wesley Willis' discography. More on that later.

Finally, the resulting dataframes are stored in hdf format. This is much faster than csv and can handle certain object columns that csv cannot.

In [4]:
def scrape_all_playlists( auth_dict ):
     
    #Obtain Spotify auth token    
    sp = spotify_auth( auth_dict )
    #If no token obtained, return.
    if sp is None:
        return

    #Tracks from my personal library that I managed to find on Spotify
    df_nix = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='1ypVHzjgXq07PwQR7SBcGU')

    #Wesley Willis' discography on Spotify.
    df_ww = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='5IElZsqCmr3gilbBSLLl78')

    #Around 90% of Billboard Hot 100 from 1960-2016. Split by decade 
    #(approximately 1000 tracks each).
    df_10 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='0Nqyr374lO74EpslPB42tW')
    df_00 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='0h8U2fFoxj4s2f67reBIc1')
    df_90 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='68yLpbpfFWs4pyqaPbfIpi')
    df_80 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='0eRDNZ0O3h1P2FRQYoy4CC')
    df_70 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='32CwVeir8S150tjZqWd4Od')
    df_60 = scrape_playlist_dataframe(
        sp, username='nxbrnt', playlist_id='0R2UZF1KlHgbw3WKTD1KDm')

    #Add decade column to Billboard dataframes, and 
    #concatenate them into one dataframe.
    df_10['decade'] = 2010
    df_00['decade'] = 2000
    df_90['decade'] = 1990
    df_80['decade'] = 1980
    df_70['decade'] = 1970
    df_60['decade'] = 1960
    df_top = pd.concat([df_10,df_00,df_90,df_80,df_70,df_60]).reset_index(
        drop=True)

    #Add dataset indicator column, and concatenate df_nix and df_top
    df_nix['ds']='nix'
    df_top['ds']='top'
    df = pd.concat([df_nix,df_top]).reset_index(drop=True)

    #Construct a uniform variable to use as 
    #a dummy in certain plot functions
    df[''] = ''
    df_ww[''] = ''

    #Pull out (currently) unused columns into separate dataframes
    cols = ['available_markets','preview_url','images','images_album']
    df_extra = df[cols]
    df.drop(cols, axis=1, inplace=True)
    df_ww_extra = df_ww[cols]
    df_ww.drop(cols, axis=1, inplace=True)

    #Write dataframes to hdf. Much faster than csv, 
    #and csv screws up certain object columns.
    df.to_hdf('./data/df.h5', 'main')
    df_extra.to_hdf('./data/df.h5', 'extra')
    df_ww.to_hdf('./data/df_ww.h5', 'main')
    df_ww_extra.to_hdf('./data/df_ww.h5', 'extra')
    
    return df, df_ww

# Scraping and Loading our Datasets

From this point on, we will simply load our dataframes from their respective h5 files. We can uncomment the code at the top of the following cell when we want to re-scrape our datasets. Make sure to include valid API keys either here, in `keys.py`, or in `secret_keys.py`.

In [2]:
# #Only uncomment and run this when re-scraping datasets
# auth_dict = { 'client_id' : client_id,
#               'secret' : secret,
#               'redirect_uri' : redirect_uri,
#               'scope' : 'user-library-read',
#               'username' : username }
# df,df_ww = scrape_all_playlists(auth_dict);

df = pd.read_hdf('./data/df.h5', 'main')
df_ww = pd.read_hdf('./data/df_ww.h5', 'main')

Let's have a look at our dataset!

In [3]:
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11540 entries, 0 to 11539
Data columns (total 34 columns):
acousticness         11539 non-null float64
album_type           11540 non-null object
danceability         11539 non-null float64
decade               5320 non-null float64
disc_number          11540 non-null int64
ds                   11540 non-null object
duration_mins        11539 non-null float64
energy               11539 non-null float64
explicit             11540 non-null bool
genres               11540 non-null object
id                   11540 non-null object
id_artist            11540 non-null object
instrumentalness     11539 non-null float64
key                  11539 non-null float64
label                11540 non-null object
liveness             11539 non-null float64
loudness             11539 non-null float64
mode                 11539 non-null float64
name                 11540 non-null object
name_album           11540 non-null object
name_artist          1154

,acousticness,album_type,danceability,decade,disc_number,ds,duration_mins,energy,explicit,genres,id,id_artist,instrumentalness,key,label,liveness,loudness,mode,name,name_album,name_artist,num_followers,popularity,popularity_album,popularity_artist,release_day,release_month,release_year,speechiness,tempo,time_signature,track_number,valence,
3613,0.000844,album,0.479,NaN,1,nix,5.818433,0.9950,False,[Power Noise],52e0nh25QEkGAquu2Rvkyq,6NmVthYxYQNCeKst1we0fz,0.268,9.0,HANDS,0.6120,-6.654,1.0,Where the Deep Grass Grows,Oto,Mono No Aware,624,1,4,3,3.0,11.0,2017,0.2710,137.982,4.0,2,0.1320,
2481,0.972000,album,0.363,NaN,1,nix,5.654000,0.0492,False,"[Art Rock, Dance Rock, Electronic, Industrial,...",2spKqhUKZodGKhdwShWnRv,5KQMtyPE8DCQNUzoNqlEsE,0.200,5.0,Beggars Banquet,0.0968,-21.256,1.0,Please Push No More,Telekon,Gary Numan,104713,17,37,56,NaN,NaN,1979,0.0375,182.826,4.0,10,0.2720,
2296,0.009330,single,0.565,NaN,1,nix,3.093750,0.8730,False,"[Bass Music, Electronic Trap, Glitch Hop, Vapo...",0hiiRAYYxZxJRnaQwGDl3a,2c2X6dr1PHctH24BxNOjHi,0.000,5.0,EPROM,0.3590,-6.151,0.0,Brixton,Samurai,Eprom,18883,19,33,44,29.0,7.0,2016,0.7020,128.364,4.0,3,0.3540,
3147,0.000009,album,0.484,NaN,1,nix,6.391783,0.6780,False,"[Aggrotech, Dark Wave, Ebm, Electro-Industrial...",3dD7I8fpBTd1LFSZhKim5P,0klcoRwPQF1GMv8FrA7F8V,0.884,2.0,Cleopatra Records,0.1310,-14.570,1.0,Prying Eyes,Underneath The Laughter,Leæther Strip,5706,1,6,28,NaN,NaN,1994,0.0285,131.632,4.0,6,0.0482,
8577,0.000889,album,0.671,1990.0,1,top,3.978450,0.8620,False,"[Bubblegum Dance, Eurodance, Europop, German T...",13vDH7LlES2hajU7yR07Mz,2vRfKzjQYJQd67X8x49MOh,0.700,5.0,Hansa,0.1670,-11.367,1.0,One More Time,One More Time,Real McCoy,26883,32,29,51,25.0,3.0,1997,0.0351,133.117,3.0,1,0.8550,


Looks good!

# End
Now... Let's explore the data! [Part 2: Basic Exploration](spotify_2_basic_exploration.ipynb)

---

| [Main](index.ipynb) | [Part 2: Basic Exploration](spotify_2_basic_exploration.ipynb) >